__Objective__: Analyse error bands computed in 46a

__Runtime__: CPU

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!git clone https://github.com/utkarsh512/CreateDebateScraper.git

In [ ]:
%cd CreateDebateScraper/src/nested/

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm 
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib import colors
from thread import Thread, Comment # from CreateDebateScraper
import pickle
from copy import deepcopy

In [ ]:
comments = dict()

# topical forums on CreateDebate
categories = ['business', 'comedy', 'entertainment', 'health', 'law', 'nsfw',
              'politics2', 'religion', 'science', 'shopping', 'sports',
              'technology', 'travel', 'world']

# topical forums we're interested in!
categories_selected = ['politics2', 'religion', 'world', 'science', 'law', 'technology']
categories_labels = ['politics', 'religion', 'world', 'science', 'law', 'technology']

for x in categories_selected:
    comments[x] = list()

In [ ]:
# loading dataset from drive for interesting topical forums
for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()
    #print(f'{cat} - {len(threads)}')
    authors = dict()
    for thread in threads:
        for k, v in thread.comments.items():
            try:
                authors[v.author].append(v)
            except:
                authors[v.author] = list()
                authors[v.author].append(v)
    ctr = 0
    with open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/comments_with_score.log', 'rb') as fp:
        cws = pickle.load(fp)
    for author in authors.keys():
        for i in range(len(authors[author])):
            comment = authors[author][i]
            foo = deepcopy(comment.__dict__)
            foo['tag'] = cat
            #foo['score'] = cws[ctr][0]
            #foo['validation'] = cws[ctr][1][0]
            comments[cat].append(foo)
            ctr += 1

In [ ]:
all_c = list()

for cat in categories_selected:
    for comment in comments[cat]:
        all_c.append(comment)

In [ ]:
def load_file(file_id):
    with open(f'/content/gdrive/MyDrive/DL/CreateDebate/errorband/label{file_id}.log', 'rb') as f:
        obj = pickle.load(f)
    return obj

In [ ]:
labels = [load_file(i) for i in range(10)]

In [ ]:
for i, comment in enumerate(all_c):
    for j in range(10):
        comment[f'bert_label_{j}'] = labels[j][i]['label']

In [ ]:
df = pd.DataFrame(all_c)

In [ ]:
df

In [ ]:
cdf = list()

for cat in categories_selected:
    cur_df = df[df['tag']==cat]
    cdf.append(cur_df)

In [ ]:
for i, cat in enumerate(categories_selected):
    ah_p = list()
    for j in range(10):
        foo = list(cdf[i][f'bert_label_{j}'])
        res = foo.count('LABEL_1') / len(foo) * 100
        ah_p.append(res)
    print(f'{cat}\t{np.average(ah_p)}\t{np.std(ah_p)}')

In [ ]:
ah_p = list()
for j in range(10):
    foo = list(df[f'bert_label_{j}'])
    res = foo.count('LABEL_1') / len(foo) * 100
    ah_p.append(res)
print(f'overall\t{np.average(ah_p)}\t{np.std(ah_p)}')